In [41]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.vision import *
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from fastai.vision.image import Image

BATCH_SIZE = 64 #smaller batch size is better for training, but may take longer
IMG_SIZE = 256

In [57]:
# Making pretrained weights work without needing to find the default filename
if not os.path.exists('/tmp/.cache/torch/checkpoints/'):
        os.makedirs('/tmp/.cache/torch/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/tmp/.cache/torch/checkpoints/resnet50-19c8e357.pth'


os.listdir('../input')

['aptos2019-blindness-detection', 'resnet50']

In [7]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
seed_everything(SEED)

In [8]:
base_image_dir = os.path.join('..', 'input/aptos2019-blindness-detection/')
train_dir = os.path.join(base_image_dir,'train_images/')
df = pd.read_csv(os.path.join(base_image_dir, 'train.csv'))
df['path'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))
df = df.drop(columns=['id_code'])
df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
df.head(10)

,diagnosis,path
0,0,../input/aptos2019-blindness-detection/train_i...
1,1,../input/aptos2019-blindness-detection/train_i...
2,3,../input/aptos2019-blindness-detection/train_i...
3,4,../input/aptos2019-blindness-detection/train_i...
4,0,../input/aptos2019-blindness-detection/train_i...
5,0,../input/aptos2019-blindness-detection/train_i...
6,0,../input/aptos2019-blindness-detection/train_i...
7,4,../input/aptos2019-blindness-detection/train_i...
8,2,../input/aptos2019-blindness-detection/train_i...
9,2,../input/aptos2019-blindness-detection/train_i...


# Splitting dataset into train+val / test - 60+20 / 20

In [9]:
from sklearn.model_selection import train_test_split

# Perform train-validation split with an 80-20 ratio
train_df, val_df = train_test_split(df, test_size=0.2)

# Reset the indices of the resulting train and validation dataframes
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

train_df.head(10)

,diagnosis,path
0,3,../input/aptos2019-blindness-detection/train_i...
1,0,../input/aptos2019-blindness-detection/train_i...
2,2,../input/aptos2019-blindness-detection/train_i...
3,0,../input/aptos2019-blindness-detection/train_i...
4,2,../input/aptos2019-blindness-detection/train_i...
5,0,../input/aptos2019-blindness-detection/train_i...
6,0,../input/aptos2019-blindness-detection/train_i...
7,0,../input/aptos2019-blindness-detection/train_i...
8,1,../input/aptos2019-blindness-detection/train_i...
9,1,../input/aptos2019-blindness-detection/train_i...


In [47]:
def preprocess(img):
    # Cropping the image
    img = np.array(img.data)
    img = np.transpose(img, (1, 2, 0))   
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray_img = np.round(gray_img * 255).astype(np.uint8)
    mask = gray_img>6
    
    check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
    if (check_shape == 0): 
        print("Image is too dark. We would crop out everything.")
        return img # return original image
    else:
        img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
        img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
        img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
        
        img = np.stack([img1,img2,img3],axis=-1)

    # Highlight relevant regions
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = np.round(img * 255).astype(np.uint8)
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0,0), 10), -4, 128)
    
    # Print the type of the converted Image object
    print("Image type:", type(img))
    # tensor_transform(img)
    img = Image(img)
    
    return img

In [51]:
tfms = get_transforms(do_flip=True,flip_vert=True,max_rotate=360,max_warp=0,max_zoom=1.1,max_lighting=0.1,p_lighting=0.5)
src = (ImageList.from_df(df=train_df,path='./',cols='path') #get dataset from dataset
        .split_by_rand_pct(0.25) #Splitting the dataset into 60/20
        .label_from_df(cols='diagnosis',label_cls=FloatList) #obtain labels from the level column
      )
data= (src.transform(tfms,size=IMG_SIZE,resize_method=ResizeMethod.SQUISH,padding_mode='zeros') #Data augmentation
        .databunch(bs=BATCH_SIZE,num_workers=4) #DataBunch    
       )

In [50]:
# Preprocess images in the train dataset
#for i in range(len(data.train_ds)):
#    img = data.train_ds.x[i] # Access the Image object
#    img = preprocess(img) # Preprocess the image
#    data.train_ds.x[i] = img # Assign the preprocessed image back to the dataset

# Preprocess images in the validation dataset
#for i in range(len(data.valid_ds)):
#    img = data.valid_ds.x[i] # Access the Image object
#    img = preprocess(img) # Preprocess the image
#    data.valid_ds.x[i] = img # Assign the preprocessed image back to the dataset

## Training (Transfer learning)

The Cohen's quadratically weighted kappa is a better metric when dealing with imbalanced datasets like this one, and for measuring inter-rater agreement for categorical classification (the raters being the human-labeled dataset and the neural network predictions). Here is an implementation based on the scikit-learn's implementation, but converted to a pytorch tensor, as that is what fastai uses.

In [52]:
from sklearn.metrics import cohen_kappa_score
def quadratic_kappa(y_hat, y):
    return torch.tensor(cohen_kappa_score(torch.round(y_hat), y, weights='quadratic'),device='cuda:0')

**Training:**

We use transfer learning, where we retrain the last layers of a pretrained neural network. I use the ResNet50 architecture trained on the ImageNet dataset, which has been commonly used for pre-training applications in computer vision. Fastai makes it quite simple to create a model and train:

In [60]:
from torchvision.models import resnet50

model = cnn_learner(data, resnet50, metrics=[quadratic_kappa], pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

Here we can see that the loss decreases fastest around `lr=1e-2` so that is what we will use to train:

In [ ]:
learn.fit_one_cycle(4,max_lr = 1e-2)

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
learn.fit_one_cycle(6, max_lr=slice(1e-6,1e-3))

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
learn.export()
learn.save('stage-2')

Let's evaluate our model:

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
#interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

## Optimize the Metric

Optimizing the quadratic kappa metric was an important part of the top solutions in the previous competition. Thankfully, @abhishek has already provided code to do this for us. We will use this to improve the score.

In [ ]:
valid_preds = learn.get_preds(ds_type=DatasetType.Valid)

In [ ]:
import numpy as np
import pandas as pd
import os
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json

In [ ]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
        print(-loss_partial(self.coef_['x']))

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [ ]:
optR = OptimizedRounder()
optR.fit(valid_preds[0],valid_preds[1])

In [ ]:
coefficients = optR.coefficients()
print(coefficients)

## TTA

Test-time augmentation, or TTA, is a commonly-used technique to provide a boost in your score, and is very simple to implement. Fastai already has TTA implemented, but it is not the best for all purposes, so I am redefining the fastai function and using my custom version.

In [ ]:
from fastai.core import *
from fastai.basic_data import *
from fastai.basic_train import *
from fastai.torch_core import *
def _tta_only(learn:Learner, ds_type:DatasetType=DatasetType.Valid, num_pred:int=10) -> Iterator[List[Tensor]]:
    "Computes the outputs for several augmented inputs for TTA"
    dl = learn.dl(ds_type)
    ds = dl.dataset
    old = ds.tfms
    aug_tfms = [o for o in learn.data.train_ds.tfms]
    try:
        pbar = master_bar(range(num_pred))
        for i in pbar:
            ds.tfms = aug_tfms
            yield get_preds(learn.model, dl, pbar=pbar)[0]
    finally: ds.tfms = old

Learner.tta_only = _tta_only

def _TTA(learn:Learner, beta:float=0, ds_type:DatasetType=DatasetType.Valid, num_pred:int=10, with_loss:bool=False) -> Tensors:
    "Applies TTA to predict on `ds_type` dataset."
    preds,y = learn.get_preds(ds_type)
    all_preds = list(learn.tta_only(ds_type=ds_type, num_pred=num_pred))
    avg_preds = torch.stack(all_preds).mean(0)
    if beta is None: return preds,avg_preds,y
    else:            
        final_preds = preds*beta + avg_preds*(1-beta)
        if with_loss: 
            with NoneReduceOnCPU(learn.loss_func) as lf: loss = lf(final_preds, y)
            return final_preds, y, loss
        return final_preds, y

Learner.TTA = _TTA

## Submission
Let's now create a submission

In [ ]:
sample_df = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
sample_df.head()

In [ ]:
learn.data.add_test(ImageList.from_df(sample_df,'../input/aptos2019-blindness-detection',folder='test_images',suffix='.png'))

In [ ]:
preds,y = learn.TTA(ds_type=DatasetType.Test)

In [ ]:
test_predictions = optR.predict(preds, coefficients)

In [ ]:
sample_df.diagnosis = test_predictions.astype(int)
sample_df.head()

In [ ]:
sample_df.to_csv('submission.csv',index=False)